## FEMA API workflow

In [21]:
# set up dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import cartopy
import geoviews
import pyproj
import holoviews

from scipy.stats import linregress

#no need for an api key yet, since FEMA API is open-read

In [22]:
# install folium for a more interactive mapping output
!pip install folium pandas

     ---------------------------------------- 0.0/102.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/102.3 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/102.3 kB ? eta -:--:--
     ----------- ------------------------- 30.7/102.3 kB 435.7 kB/s eta 0:00:01
     --------------------------------- --- 92.2/102.3 kB 655.4 kB/s eta 0:00:01
     ------------------------------------ 102.3/102.3 kB 657.4 kB/s eta 0:00:00


In [4]:
# set up url and API endpoint
fema_url = 'https://www.fema.gov/api/open/v2/HazardMitigationGrantProgramDisasterSummaries'
headers = {'User-Agent': 'XY', 'Content-type': 'application/json'}

In [5]:
# build request variable
response = requests.get(fema_url, headers = headers)

In [6]:
# check the status of requests to the API endpoint
response.status_code

200

In [7]:
fema_data = response.json()

In [9]:
content_type = response.headers.get('content-type', 'unknown')
print(f"Content Type: {content_type}")

Content Type: application/json; charset=utf-8


In [10]:
fema_data_df = pd.DataFrame(fema_data["HazardMitigationGrantProgramDisasterSummaries"])

In [11]:
# Display the freshly brewed DataFrame
fema_data_df.head()

,disasterNumber,declarationDate,disasterType,incidentType,title,region,state,disasterCloseoutStatus,hmgpCloseoutStatus,disasterCloseoutDate,...,obligatedRegularAmount,obligatedRecipientAdmin,obligatedSubrecipientAdmin,obligatedRecipientMgmtAmt,obligatedSubrecipMgmtAmt,pendingProjectsQuantity,pendingFedShareProposedAmt,hash,lastRefresh,id
0,4162,2014-01-23T00:00:00.000Z,DR,Severe Storm,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND FLOODING",10,Alaska,Open,Open,None,...,3529628.00,0.0,0.0,217140.0,0.0,0.0,NaN,13c5cdf3575321d46c9b633c24cf21e0e84d7c4d,2023-05-13T17:11:10.832Z,26aeb8cf-b9f5-4d68-9b59-6842d76c1510
1,1316,2000-02-17T00:00:00.000Z,DR,Severe Storm,SEVERE WINTER STORMS AND AVALANCHES,10,Alaska,Closed,Closed,2010-04-26T00:00:00.000Z,...,1428092.00,22927.0,44892.0,17067.0,0.0,0.0,NaN,a68619627e9fcc0c7a13dceea568ecb1dc8bb449,2023-05-13T17:11:10.832Z,27f80f69-2ca0-47dc-ac9f-19683141de37
2,1618,2005-12-09T00:00:00.000Z,DR,Severe Storm,"SEVERE FALL STORM, TIDAL SURGES, AND FLOODING",10,Alaska,Closed,Closed,2012-02-13T00:00:00.000Z,...,124826.00,4330.0,4362.0,22032.0,0.0,0.0,NaN,c940c03a9ee80a40bcc75a45826d4bc53f2196eb,2023-05-13T17:11:10.832Z,5b85fd56-ecf8-4b87-9ad4-cad99fe0bca8
3,4050,2011-12-22T00:00:00.000Z,DR,Severe Storm,SEVERE WINTER STORMS AND FLOODING,10,Alaska,Closed,Closed,2022-02-24T00:00:00.000Z,...,217021.00,0.0,0.0,10635.0,0.0,0.0,NaN,a4d929a1592eee59c78b6f5f51520ff861f0b963,2023-05-13T17:11:10.832Z,161b8149-e2a5-445e-bdf0-c7b0e74e5533
4,4094,2012-11-27T00:00:00.000Z,DR,Severe Storm,"SEVERE STORM, STRAIGHT-LINE WINDS, FLOODING, A...",10,Alaska,Open,Open,None,...,3341913.97,0.0,0.0,163420.0,0.0,0.0,NaN,f893a812ac93af96834fdc6a49e124f0af263bfb,2023-05-13T17:11:10.832Z,115c3171-4a6a-470b-8461-d35252544135


In [12]:
# For our project, we'll need just the reports that include Flooding in the column 'title'
# So, time to convert the first DataFrame into a cleaned df
first_flooding_df = fema_data_df[fema_data_df['title'].str.contains('Flooding', case=False)]

In [13]:
# Display first_flooding_df
first_flooding_df.head()

,disasterNumber,declarationDate,disasterType,incidentType,title,region,state,disasterCloseoutStatus,hmgpCloseoutStatus,disasterCloseoutDate,...,obligatedRegularAmount,obligatedRecipientAdmin,obligatedSubrecipientAdmin,obligatedRecipientMgmtAmt,obligatedSubrecipMgmtAmt,pendingProjectsQuantity,pendingFedShareProposedAmt,hash,lastRefresh,id
0,4162,2014-01-23T00:00:00.000Z,DR,Severe Storm,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND FLOODING",10,Alaska,Open,Open,None,...,3529628.00,0.0,0.0,217140.0,0.0,0.0,NaN,13c5cdf3575321d46c9b633c24cf21e0e84d7c4d,2023-05-13T17:11:10.832Z,26aeb8cf-b9f5-4d68-9b59-6842d76c1510
2,1618,2005-12-09T00:00:00.000Z,DR,Severe Storm,"SEVERE FALL STORM, TIDAL SURGES, AND FLOODING",10,Alaska,Closed,Closed,2012-02-13T00:00:00.000Z,...,124826.00,4330.0,4362.0,22032.0,0.0,0.0,NaN,c940c03a9ee80a40bcc75a45826d4bc53f2196eb,2023-05-13T17:11:10.832Z,5b85fd56-ecf8-4b87-9ad4-cad99fe0bca8
3,4050,2011-12-22T00:00:00.000Z,DR,Severe Storm,SEVERE WINTER STORMS AND FLOODING,10,Alaska,Closed,Closed,2022-02-24T00:00:00.000Z,...,217021.00,0.0,0.0,10635.0,0.0,0.0,NaN,a4d929a1592eee59c78b6f5f51520ff861f0b963,2023-05-13T17:11:10.832Z,161b8149-e2a5-445e-bdf0-c7b0e74e5533
4,4094,2012-11-27T00:00:00.000Z,DR,Severe Storm,"SEVERE STORM, STRAIGHT-LINE WINDS, FLOODING, A...",10,Alaska,Open,Open,None,...,3341913.97,0.0,0.0,163420.0,0.0,0.0,NaN,f893a812ac93af96834fdc6a49e124f0af263bfb,2023-05-13T17:11:10.832Z,115c3171-4a6a-470b-8461-d35252544135
5,1445,2002-12-04T00:00:00.000Z,DR,Severe Storm,"SEVERE WINTER STORMS, FLOODING, COASTAL EROSIO...",10,Alaska,Closed,Closed,2012-09-05T00:00:00.000Z,...,1551703.00,20929.0,46112.0,63683.0,0.0,0.0,NaN,f909f8ac9beb1305936bf4c492e4055f9308489b,2023-05-13T17:11:10.832Z,aaa86c9e-68d9-4343-a7eb-7b07c6cac12e


In [14]:
fema_flooding_df = first_flooding_df[first_flooding_df['state'] != 'Alaska']

In [15]:
fema_flooding_df.head()

,disasterNumber,declarationDate,disasterType,incidentType,title,region,state,disasterCloseoutStatus,hmgpCloseoutStatus,disasterCloseoutDate,...,obligatedRegularAmount,obligatedRecipientAdmin,obligatedSubrecipientAdmin,obligatedRecipientMgmtAmt,obligatedSubrecipMgmtAmt,pendingProjectsQuantity,pendingFedShareProposedAmt,hash,lastRefresh,id
24,4052,2012-02-01T00:00:00.000Z,DR,Severe Storm,"SEVERE STORMS, TORNADOES, STRAIGHT-LINE WINDS,...",4,Alabama,Closed,Closed,2017-12-18T00:00:00.000Z,...,146221.00,0.00,0.00,0.00,0.0,0.0,NaN,2afa075de2c27437445fb4df0b4f67b7a6d127da,2023-05-13T17:11:10.832Z,f0cda323-c964-4db4-a532-d401d94776fb
30,1870,2009-12-31T00:00:00.000Z,DR,Severe Storm,SEVERE STORMS AND FLOODING,4,Alabama,Closed,Closed,2018-09-04T00:00:00.000Z,...,1468084.00,0.00,0.00,151463.00,0.0,0.0,NaN,71720ab2da8b8dce17cc726bb0c412e620575431,2023-05-13T17:11:10.832Z,e547be91-e472-4092-bdbd-64b7c5987f21
33,1731,2007-10-24T00:00:00.000Z,DR,Fire,"WILDFIRES, FLOODING, MUD FLOWS, AND DEBRIS FLOWS",9,California,Open,Open,None,...,31079021.27,244216.08,699849.85,2940448.86,0.0,0.0,NaN,e267e6d998431b3aa725753670911fd3ea67cef9,2023-08-19T17:15:16.526Z,a55afd8b-2ef4-43e6-a608-2fa9a9bc417a
34,1466,2003-05-12T00:00:00.000Z,DR,Severe Storm,"SEVERE STORMS, TORNADOES, AND FLOODING",4,Alabama,Closed,Closed,2011-04-07T00:00:00.000Z,...,2564638.00,39597.00,67762.00,213524.00,0.0,0.0,NaN,13a016af17fb82d7452a459f0c3ab7ae492fbb3e,2023-05-13T17:11:10.832Z,0720c454-b184-4bcc-8604-037b9dbb92ad
35,1013,1994-03-03T00:00:00.000Z,DR,Severe Storm,"SEVERE WINTER STORMS, FREEZING, FLOODING",4,Alabama,Closed,Closed,2004-05-18T00:00:00.000Z,...,1332686.00,27453.00,44899.00,0.00,0.0,1.0,NaN,e76fe1ad5b84ac5ac7965f57079fb2074f78a49c,2023-06-30T17:09:51.946Z,b5e6c9ef-e9a0-4656-b265-ad956f6faa07


In [16]:
# Drop irrelevant columns
# Declare a variable with a list of column titles to drop
columns_to_drop = ['disasterType', 'region', 'disasterCloseoutStatus', 'hmgpCloseoutStatus', 'disasterCloseoutDate', 
                   'pendingFedShareProposedAmt', 'pendingProjectsQuantity', 'hash', 'lastRefresh', 'id', 'hmgpReconciliationDate',
                  'disasterDueDateForNewApps', 'disasterPopEndDate', 'hmgpPopEndDate', 'hmgpPopExtnDate', 'obligatedInitiativeAmount',
                   'obligatedPlanningAmount', 'obligatedInitiativeAmount', 'obligatedRegularAmount', 'obligatedRecipientAdmin',
                  'obligatedSubrecipientAdmin', 'obligatedRecipientMgmtAmt', 'obligatedSubrecipMgmtAmt', 'hmgpPopExtnNum', 'liqPeriodDate', 
                   'liqPeriodMaxDate', 'liqPeriodMaxPopNum', 'liqPeriodMaxExtDate', 'liqPeriodMaxExtNum']

# Run a .drop function using the variable of column titles to drop
fema_flood_cost = fema_flooding_df.drop(columns=columns_to_drop)

In [17]:
# Display new df
fema_flood_cost.head()

,disasterNumber,declarationDate,incidentType,title,state,lockedInCeilingAmount,mitigationDollarsAvailable,obligatedTotalAmount
24,4052,2012-02-01T00:00:00.000Z,Severe Storm,"SEVERE STORMS, TORNADOES, STRAIGHT-LINE WINDS,...",Alabama,146617.0,7566.00,146221.00
30,1870,2009-12-31T00:00:00.000Z,Severe Storm,SEVERE STORMS AND FLOODING,Alabama,3097396.0,1576515.00,1672344.00
33,1731,2007-10-24T00:00:00.000Z,Fire,"WILDFIRES, FLOODING, MUD FLOWS, AND DEBRIS FLOWS",California,42417425.0,13996680.94,39741955.06
34,1466,2003-05-12T00:00:00.000Z,Severe Storm,"SEVERE STORMS, TORNADOES, AND FLOODING",Alabama,4146632.0,2178710.00,3112794.00
35,1013,1994-03-03T00:00:00.000Z,Severe Storm,"SEVERE WINTER STORMS, FREEZING, FLOODING",Alabama,1985365.0,385010.00,1672707.00


In [18]:
# Write the cleaned df to a .csv file for work in the rest of our project
csv_file = "FEMA_Flooding_Costs.csv"

fema_flood_cost.to_csv(csv_file, index=False)

In [19]:
# Read from the .csv instead of re-running all of the above code
working_flood_df = pd.read_csv(csv_file)

In [20]:
# Mapping attempt number one


,disasterNumber,declarationDate,incidentType,title,state,lockedInCeilingAmount,mitigationDollarsAvailable,obligatedTotalAmount
0,4052,2012-02-01T00:00:00.000Z,Severe Storm,"SEVERE STORMS, TORNADOES, STRAIGHT-LINE WINDS,...",Alabama,146617.0,7566.00,146221.00
1,1870,2009-12-31T00:00:00.000Z,Severe Storm,SEVERE STORMS AND FLOODING,Alabama,3097396.0,1576515.00,1672344.00
2,1731,2007-10-24T00:00:00.000Z,Fire,"WILDFIRES, FLOODING, MUD FLOWS, AND DEBRIS FLOWS",California,42417425.0,13996680.94,39741955.06
3,1466,2003-05-12T00:00:00.000Z,Severe Storm,"SEVERE STORMS, TORNADOES, AND FLOODING",Alabama,4146632.0,2178710.00,3112794.00
4,1013,1994-03-03T00:00:00.000Z,Severe Storm,"SEVERE WINTER STORMS, FREEZING, FLOODING",Alabama,1985365.0,385010.00,1672707.00
...,...,...,...,...,...,...,...,...
449,4256,2016-02-10T00:00:00.000Z,Severe Storm,SEVERE WINTER STORMS AND FLOODING,Oklahoma,8328147.0,10400.49,8724992.51
450,1926,2010-07-26T00:00:00.000Z,Severe Storm,"SEVERE STORMS, TORNADOES, STRAIGHT-LINE WINDS,...",Oklahoma,519834.0,4175.17,541078.83
451,1384,2001-06-29T00:00:00.000Z,Severe Storm,"SEVERE STORMS, FLOODING, AND TORNADOES",Oklahoma,1001451.0,230655.00,1051510.00
452,4222,2015-05-26T00:00:00.000Z,Severe Storm,"SEVERE STORMS, TORNADOES, STRAIGHT-LINE WINDS,...",Oklahoma,12507796.0,1832894.63,11286532.37


In [23]:
import folium

# Create a map centered around the US
us_map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

In [27]:
data = working_flood_df{
    'State': ['State'],
    'Value': [obligatedTotalAmount]
}

data_df = pd.DataFrame(data)

SyntaxError: invalid syntax (3213409633.py, line 1)

In [24]:
# Loop through the DataFrame and add markers for each state
for index, row in data_df():
    state = row['state']
    value = row['value']

In [26]:
 # Create a marker for each state with its name and data value
marker = folium.Marker(
    location=[0, 0],  # You'll need to specify the actual latitude and longitude for each state
    popup=f"{state}: {value}",)

marker.add_to(us_map)

# Display the map in the notebook
us_map